In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import AllTogether as t
import seaborn as sns
%matplotlib inline

#DataSets for analysis
#source: https://hub.arcgis.com/datasets/6afda9afb9f94e27adf39d36e119eb29_2
demo_df= pd.read_csv('./Census_demographics_by_County.csv')
#source: https://hub.arcgis.com/datasets/utah::us-states
demo_state_df = pd.read_csv("./US_States.csv")
#source: https://www.kaggle.com/mikejohnsonjr/united-states-crime-rates-by-county
crime_df = pd.read_csv('./crime_data_w_population_and_crime_rate.csv')

In [2]:
crime_df.head(2)

,county_name,crime_rate_per_100000,index,EDITION,PART,IDNO,CPOPARST,CPOPCRIM,AG_ARRST,AG_OFF,...,RAPE,ROBBERY,AGASSLT,BURGLRY,LARCENY,MVTHEFT,ARSON,population,FIPS_ST,FIPS_CTY
0,"St. Louis city, MO",1791.995377,1,1,4,1612,318667,318667,15,15,...,200,1778,3609,4995,13791,3543,464,318416,29,510
1,"Crittenden County, AR",1754.914968,2,1,4,130,50717,50717,4,4,...,38,165,662,1482,1753,189,28,49746,5,35


In [3]:
demo_df.head(2)

,OBJECTID,STATE,STCN,CNTYNAME,AREALAND,AREAWATER,TOTALPOP,POP_DEN,NUM_MINORITY,PCT_MINORITY,...,EMP_STAT_NOT_IN_LF,PCT_EMP_STAT_NOT_IN_LF,Shape_Length,Shape_Area,FBPOP,EUROPE,ASIA,AFRICA,OCEANIA,AMERICAS
0,1,1,1001,Autauga County,1539584444,25773561,55136,92.753326,13103,23.764872,...,15561,36.603782,2.052620,0.150260,772.0,141.0,150.0,0.0,119.0,362.0
1,2,1,1005,Barbour County,2291820953,50864677,27119,30.647195,14614,53.888418,...,11086,50.736842,2.566611,0.223256,769.0,14.0,169.0,0.0,0.0,586.0


In [4]:
demo_state_df.head(2)

,OBJECTID_1,OBJECTID,STATE_NAME,STATE_FIPS,STATE_ABBR,SHAPE_Length,SHAPE_Area
0,1,NaN,Alabama,1,AL,20.554796,12.881818
1,2,NaN,Alaska,2,AK,318.661159,167.784052


In [5]:
#Use demo_state_df to recover the states in demo_df

#Match on the state numeric code
for name in range(len(demo_df['STATE'])):
    for stateCode in range(len(demo_state_df['STATE_FIPS'])):
        if(demo_df['STATE'][name] == demo_state_df['STATE_FIPS'][stateCode]):
            demo_df['STATE'][name] = demo_state_df['STATE_ABBR'][stateCode]

<ipython-input-5-1dfa1662d4ef>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df['STATE'][name] = demo_state_df['STATE_ABBR'][stateCode]
/home/ada/pythonJupyter/jupyterenvironment/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [6]:
#Split string on comma and turn into a list. Needed for matching later.
for name in range(len(crime_df['county_name'])):
    crime_df['county_name'][name] = crime_df['county_name'][name].split(",")
    crime_df['county_name'][name][1] = crime_df['county_name'][name][1].strip()

<ipython-input-6-3bc2aae1c169>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_df['county_name'][name] = crime_df['county_name'][name].split(",")


In [7]:
# Add columns for matching
crime_df.insert(0,"JoinColumn", [np.nan] * crime_df.shape[0], True )
demo_df.insert(0,"JoinColumn", [np.nan] * demo_df.shape[0], True )

In [8]:
# Match based on the county name and the state
numEqual = 0
for x in range(len(crime_df['county_name'])):
    for y in range(len(demo_df['CNTYNAME'])):
        if crime_df['county_name'][x][0] == demo_df['CNTYNAME'][y]:
            if(crime_df['county_name'][x][1] == demo_df['STATE'][y]):
                numEqual += 1
                crime_df['JoinColumn'][x] = numEqual
                demo_df['JoinColumn'][y] = numEqual

<ipython-input-8-bb558d746821>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_df['JoinColumn'][x] = numEqual
<ipython-input-8-bb558d746821>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_df['JoinColumn'][y] = numEqual


In [9]:
#Show the number of matches                
print(numEqual)
print(crime_df.shape)
print(demo_df.shape)
#Drop Columns that did not have a match
crime_df_dropped = crime_df.dropna(subset=['JoinColumn'], how = 'any')
demo_df_dropped = demo_df.dropna(subset=['JoinColumn'], how = 'any')
#Show the number of matches in each data frame
#Note that they are the same size, there were not any duplicate matches
print(crime_df_dropped.shape)
print(demo_df_dropped.shape)

3135
(3136, 25)
(3220, 267)
(3135, 25)
(3135, 267)


In [10]:

#Columns that are not to be used in the modeling and will be dropped.
dropList = ['JoinColumn','county_name', 'EDITION', 'PART', 'IDNO', 'INDEX', 'FIPS_ST', 'FIPS_CTY','OBJECTID', 'STATE', 'STCN', 'CNTYNAME']

#Join on the matches
joinedDF = crime_df_dropped.merge(demo_df_dropped, on='JoinColumn')

#Subset by county based on potential living city locations
columbia = joinedDF[joinedDF['STATE'] =='MD'].copy()
columbia = columbia[columbia['CNTYNAME'] == "Howard County"]

odenton = joinedDF[joinedDF['STATE'] =='MD'].copy()
odenton = odenton[odenton['CNTYNAME'] == "Anne Arundel County"]

glen_burnie = joinedDF[joinedDF['STATE'] =='MD'].copy()
glen_burnie = glen_burnie[glen_burnie['CNTYNAME'] == "Anne Arundel County"]

hanover = joinedDF[joinedDF['STATE'] =='MD'].copy()
hanover = hanover[hanover['CNTYNAME'] == "Anne Arundel County"]

#Drop unneccessary columns
columbia = columbia.drop(dropList, axis =1)
odenton = odenton.drop(dropList, axis =1)
glen_burnie = glen_burnie.drop(dropList, axis =1)
hanover = hanover.drop(dropList, axis =1)
joinedDF = joinedDF.drop(dropList, axis =1)

# Population in 2019
# Used for imputation
howardCountyPop = 325690
columbiaPop = 103991
anneArundelPop = 579234
odentonPop = 41846
glen_burniePop = 69872
hanoverPop = 12952

def imputeSubset(df, portion):
    for column in df.columns:
        if np.issubdtype(df.dtypes[column], np.number):
                df[column] = df[column] * portion
                
#Impute values for the cities from their county
imputeSubset(columbia, columbiaPop / howardCountyPop)
imputeSubset(odenton, odentonPop / anneArundelPop)
imputeSubset(glen_burnie, glen_burniePop / anneArundelPop)
imputeSubset(hanover, hanoverPop / anneArundelPop)

In [14]:
def model(parameter):
    X = joinedDF.drop([parameter], axis = 1)
    y = joinedDF[parameter]
    #Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state=101011)
    # train
    lm_model = LinearRegression(normalize=True) # Instantiate
    lm_model.fit(X_train, y_train) #Fit
    # predict
    y_test_preds = lm_model.predict(X_test)
    y_train_preds = lm_model.predict(X_train)
    #score
    test_score = r2_score(y_test, y_test_preds)
    train_score = r2_score(y_train, y_train_preds)
    print("The rsquared on the training data was {}.  The rsquared on the test data was {}.".format(train_score, test_score))
    
    coefs_df = pd.DataFrame()
    coefs_df['est_int'] = X_train.columns
    coefs_df['coefs'] = lm_model.coef_
    coefs_df['abs_coefs'] = np.abs(lm_model.coef_)
    coefs_df = coefs_df.sort_values('abs_coefs', ascending=False)
    
    
    columbiaDropped = columbia.drop([parameter], axis = 1)
    odentonDropped =odenton.drop([parameter], axis = 1)
    glen_burnieDropped = glen_burnie.drop([parameter], axis = 1)
    hanoverDropped = hanover.drop([parameter], axis = 1)
    
    columbiaVal = lm_model.predict(columbiaDropped)[0] / columbiaPop
    odentonVal = lm_model.predict(odentonDropped)[0] / odentonPop
    glen_burnieVal = lm_model.predict(glen_burnieDropped)[0] / glen_burniePop
    hanoverVal = lm_model.predict(hanoverDropped)[0] / hanoverPop
    
    
    
    return [[columbiaVal, odentonVal,glen_burnieVal,hanoverVal], coefs_df] 

In [15]:
results = pd.DataFrame({'City' : ["Columbia", "Odenton", "Glen Burnie", "Hanover"]})
results.set_index('City')
print(results)

          City
0     Columbia
1      Odenton
2  Glen Burnie
3      Hanover


In [16]:
result = model("RAPE")
results.insert(1, "RAPE",result[0])
result[1].head(20)

The rsquared on the training data was 0.9737776942603972.  The rsquared on the test data was 0.7162012091176642.


,est_int,coefs,abs_coefs
269,PCT_EMP_STAT_NOT_IN_LF,1.251427e+12,1.251427e+12
259,PCT_EMP_STAT_IN_LF,1.251427e+12,1.251427e+12
171,PCT_HLI_IE_LI,1.507803e+11,1.507803e+11
173,PCT_HLI_IE_NLI,9.297506e+10,9.297506e+10
59,PCT_AGE_GT17,-8.781943e+10,8.781943e+10
53,PCT_AGE_LT18,-8.781943e+10,8.781943e+10
169,PCT_HLI_IE,-8.753956e+10,8.753956e+10
181,PCT_HLI_OTHER,8.232970e+10,8.232970e+10
185,PCT_HLI_OTHER_NLI,-7.689419e+10,7.689419e+10
148,PCT_LAN_ENG_W,-6.921693e+10,6.921693e+10


In [17]:
result = model("ARSON")
results.insert(1, "ARSON",result[0])
result[1].head(20)

The rsquared on the training data was 0.9539673580763047.  The rsquared on the test data was 0.3397538395428641.


,est_int,coefs,abs_coefs
259,PCT_EMP_STAT_IN_LF,-1.740746e+12,1.740746e+12
269,PCT_EMP_STAT_NOT_IN_LF,-1.740746e+12,1.740746e+12
179,PCT_HLI_API_NLI,2.412376e+11,2.412376e+11
175,PCT_HLI_API,-2.234421e+11,2.234421e+11
171,PCT_HLI_IE_LI,-1.938352e+11,1.938352e+11
183,PCT_HLI_OTHER_LI,-1.578975e+11,1.578975e+11
177,PCT_HLI_API_LI,1.282647e+11,1.282647e+11
159,PCT_LINGISO,1.129729e+11,1.129729e+11
169,PCT_HLI_IE,9.865786e+10,9.865786e+10
173,PCT_HLI_IE_NLI,-8.086235e+10,8.086235e+10


In [18]:
result = model("MVTHEFT")
results.insert(1, "MVTHEFT",result[0])
result[1].head(20)

The rsquared on the training data was 0.9999999521064714.  The rsquared on the test data was 0.9999998983422953.


,est_int,coefs,abs_coefs
269,PCT_EMP_STAT_NOT_IN_LF,9.995406e+09,9.995406e+09
259,PCT_EMP_STAT_IN_LF,9.995406e+09,9.995406e+09
177,PCT_HLI_API_LI,-3.694731e+09,3.694731e+09
179,PCT_HLI_API_NLI,-3.188915e+09,3.188915e+09
175,PCT_HLI_API,3.086010e+09,3.086010e+09
59,PCT_AGE_GT17,2.967552e+09,2.967552e+09
53,PCT_AGE_LT18,2.967552e+09,2.967552e+09
183,PCT_HLI_OTHER_LI,-1.833522e+09,1.833522e+09
169,PCT_HLI_IE,-1.369639e+09,1.369639e+09
152,PCT_LAN_ENG_NA,1.360300e+09,1.360300e+09


In [19]:
result = model("LARCENY")
results.insert(1, "LARCENY",result[0])
result[1].head(20)

The rsquared on the training data was 0.9999999979573293.  The rsquared on the test data was 0.999999997083696.


,est_int,coefs,abs_coefs
269,PCT_EMP_STAT_NOT_IN_LF,1.021766e+10,1.021766e+10
259,PCT_EMP_STAT_IN_LF,1.021766e+10,1.021766e+10
177,PCT_HLI_API_LI,-3.750320e+09,3.750320e+09
179,PCT_HLI_API_NLI,-3.239929e+09,3.239929e+09
175,PCT_HLI_API,3.136848e+09,3.136848e+09
59,PCT_AGE_GT17,2.973672e+09,2.973672e+09
53,PCT_AGE_LT18,2.973672e+09,2.973672e+09
183,PCT_HLI_OTHER_LI,-1.840216e+09,1.840216e+09
169,PCT_HLI_IE,-1.367656e+09,1.367656e+09
152,PCT_LAN_ENG_NA,1.353940e+09,1.353940e+09


In [20]:
result = model("BURGLRY")
results.insert(1, "BURGLRY",result[0])
result[1].head(20)

The rsquared on the training data was 0.9999999816071707.  The rsquared on the test data was 0.9999999793052033.


,est_int,coefs,abs_coefs
269,PCT_EMP_STAT_NOT_IN_LF,1.030303e+10,1.030303e+10
259,PCT_EMP_STAT_IN_LF,1.030303e+10,1.030303e+10
177,PCT_HLI_API_LI,-3.760505e+09,3.760505e+09
179,PCT_HLI_API_NLI,-3.250798e+09,3.250798e+09
175,PCT_HLI_API,3.147183e+09,3.147183e+09
59,PCT_AGE_GT17,2.969849e+09,2.969849e+09
53,PCT_AGE_LT18,2.969849e+09,2.969849e+09
61,PCT_MALES,-1.885949e+09,1.885949e+09
63,PCT_FEMALES,-1.885949e+09,1.885949e+09
183,PCT_HLI_OTHER_LI,-1.833829e+09,1.833829e+09


In [21]:
result = model("AGASSLT")
results.insert(1, "AGASSLT",result[0])
result[1].head(20)

The rsquared on the training data was 0.9858276515273506.  The rsquared on the test data was 0.7630907938511257.


,est_int,coefs,abs_coefs
269,PCT_EMP_STAT_NOT_IN_LF,-6.239436e+12,6.239436e+12
259,PCT_EMP_STAT_IN_LF,-6.239436e+12,6.239436e+12
59,PCT_AGE_GT17,-1.605918e+12,1.605918e+12
53,PCT_AGE_LT18,-1.605918e+12,1.605918e+12
183,PCT_HLI_OTHER_LI,1.012390e+12,1.012390e+12
63,PCT_FEMALES,-9.243892e+11,9.243892e+11
61,PCT_MALES,-9.243892e+11,9.243892e+11
185,PCT_HLI_OTHER_NLI,7.759634e+11,7.759634e+11
181,PCT_HLI_OTHER,-6.766975e+11,6.766975e+11
169,PCT_HLI_IE,5.823059e+11,5.823059e+11


In [22]:
result = model("ROBBERY")
results.insert(1, "ROBBERY",result[0])
result[1].head(20)

The rsquared on the training data was 0.9969916270011239.  The rsquared on the test data was 0.823755330025302.


,est_int,coefs,abs_coefs
269,PCT_EMP_STAT_NOT_IN_LF,-1.141678e+12,1.141678e+12
259,PCT_EMP_STAT_IN_LF,-1.141678e+12,1.141678e+12
177,PCT_HLI_API_LI,-4.676747e+11,4.676747e+11
171,PCT_HLI_IE_LI,-3.980697e+11,3.980697e+11
183,PCT_HLI_OTHER_LI,-3.858850e+11,3.858850e+11
61,PCT_MALES,3.755275e+11,3.755275e+11
63,PCT_FEMALES,3.755275e+11,3.755275e+11
165,PCT_HLI_SPANISH_LI,-3.138486e+11,3.138486e+11
159,PCT_LINGISO,2.749814e+11,2.749814e+11
154,PCT_LAN_ENG_LTW,2.439053e+11,2.439053e+11


In [ ]:
results.hist()

In [ ]:
snprint(results)

In [ ]:
results.sum(axis= 1)